# Consultas Integradas

Este notebook tiene el fin de generar las consultas planteadas en el trabajo práctico integrador

#### Importo librerias

In [116]:
# Limpia todas las variables existentes
%reset -f

# Habilita autoreload para recargar automáticamente todos los módulos
%load_ext autoreload
%autoreload 2

In [117]:
import os
from pathlib import Path
import pandas as pd
from pymongo.errors import ConnectionFailure

from db_connections import client, db_neo4j, db_redis
from src import mongo, neo4j, utils, redis

In [118]:
# =====================
# TEST DE CONEXIONES
# =====================

#Conecto con MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Conecto con Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Conecto con Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

## A.Mostrar los usuarios que visitaron “Bariloche”. 

In [119]:
query = """ 
MATCH (U:Usuario)-[:VISITO]->(D:Destino)
WHERE D.ciudad='La Plata'
RETURN DISTINCT U.usuario_id AS id, U.nombre AS Nombre, U.apellido AS Apellido
"""

usuarios = neo4j.consulta(db_neo4j, query)
usuarios

### B.Mostrar los amigos de Juan que visitaron algún destino que visitó él, mostrar el nombre del Usuario y el destino. 

In [120]:
import pandas as pd

# Pedimos el nombre del usuario
nombre_usuario = input("Introduce tu nombre para saber qué lugares visitaste con tus amigos: ")

# Query Cypher corregida: el parámetro se llama $nombre
query = """
MATCH (u:Usuario {nombre: $nombre})-[:AMIGO_DE]-(amigo:Usuario)
MATCH (u)-[:VISITO]->(d:Destino)<-[:VISITO]-(amigo)
RETURN 
    amigo.nombre AS Nombre,
    collect(DISTINCT d.ciudad) AS Destinos_Compartidos
ORDER BY Nombre
"""

# Mensaje de encabezado
print("\n" + "-"*60)
print(f"{nombre_usuario.upper()} TUS AMIGOS VISITARON ESTOS MISMOS DESTINOS QUE TÚ")
print("-"*60 + "\n")

# Ejecutamos la consulta usando la función consulta() que devuelve un DataFrame
usuarios = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

# Mostramos el resultado
if usuarios.empty:
    print("No se encontraron amigos que hayan visitado los mismos destinos.")
else:
    print(usuarios)


### C. Sugerir destinos a un usuario que no haya visitado él ni sus amigos. 

In [121]:
nombre_usuario = input("Introduce tu nombre para saber qué lugares nuevos que no visitaste ni tu ni tus amigos: ")

query = """
MATCH (d:Destino)
WHERE 
NOT EXISTS {MATCH (d)<-[:VISITO]-(u:Usuario {nombre: $nombre})}
AND 
NOT EXISTS {MATCH (d)<-[:VISITO]-(:Usuario)-[:AMIGO_DE]-(:Usuario {nombre: $nombre})}
RETURN DISTINCT d.ciudad AS Destinos_No_Visitados
ORDER BY d.ciudad
"""

print("\n" + "-"*60)
print(f"DESTINOS NUEVOS PARA {nombre_usuario.upper()} Y SUS AMIGOS")
print("-"*60 + "\n")

destinos = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

if destinos.empty:
    print("No hay destinos nuevos disponibles.")
else:
    # Mostrar cada destino en lista
    for d in destinos['Destinos_No_Visitados']:
        print(f"- {d}")



## d. Recomendar destinos basados en viajes de amigos.

In [122]:
nombre_usuario = input("Introduce tu nombre para saber qué lugares te recomendamos: ")

query = """
MATCH (u:Usuario {nombre:$nombre})-[:AMIGO_DE]-(amigo:Usuario)
MATCH (amigo)-[:VISITO]->(d:Destino)
WHERE NOT (u)-[:VISITO]->(d)   
RETURN DISTINCT d.ciudad AS Destino_Recomendado
ORDER BY d.ciudad
"""

print("\n" + "-"*60)
print(f"RECOMENDACIONES PARA {nombre_usuario.upper()} EN FUNCIÓN DE SUS AMIGOS")
print("-"*60 + "\n")

destinos = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

if destinos.empty:
    print("No hay destinos recomendados nuevos para vos.")
else:
    for d in destinos['Destino_Recomendado']:
        print(f"- {d}")


## e. Listar los hoteles en los destinos recomendados del punto anterior. 

In [123]:
nombre_base = "clase"
coleccion = "hoteles"

# Lista de destinos recomendados
lista_destinos = destinos["Destino_Recomendado"].dropna().unique().tolist()

if not lista_destinos:
    print("No hay destinos recomendados disponibles.")
else:
    filtro = {"ciudad": {"$in": lista_destinos}}
    proyeccion = {"_id": 0, "nombre": 1, "ciudad": 1, "direccion": 1}

    # Obtener datos de Mongo y convertir a DataFrame
    cursor = mongo.obtener_cursor(
        nombre_base=nombre_base,
        nombre_coleccion=coleccion,
        filtro=filtro,
        proyeccion=proyeccion
    )

    hoteles = pd.DataFrame(list(cursor))

    if hoteles.empty:
        print("No se encontraron hoteles en los destinos recomendados.")
    else:
        # Ordenar por ciudad y luego por nombre de hotel
        hoteles.sort_values(by=["ciudad", "nombre"], inplace=True)
        print(hoteles.to_string(index=False))
       



## f.Ver las reservas en proceso, es decir que aún no están concretadas. 

## g. Listar los usuarios conectados actualmente. 

###### h. Mostrar los destinos con precio inferior a $100.000

## i. Mostrar todos los Hoteles de “Jujuy”.

In [18]:
nombre_base = "clase"
coleccion = "hoteles"

# Lista de destinos recomendados
ciudad = ["San Salvador de Jujuy"]

if not lista_destinos:
    print("No hay destinos recomendados disponibles.")
else:
    filtro = {"ciudad": {"$in": ciudad}}
    proyeccion = {"_id": 0, "nombre": 1, "ciudad": 1, "direccion": 1}

    # Obtener datos de Mongo y convertir a DataFrame
    cursor = mongo.obtener_cursor(
        nombre_base=nombre_base,
        nombre_coleccion=coleccion,
        filtro=filtro,
        proyeccion=proyeccion
    )

    hoteles = pd.DataFrame(list(cursor))

    if hoteles.empty:
        print("No se encontraron hoteles en los destinos recomendados.")
    else:
        # Ordenar por ciudad y luego por nombre de hotel
        hoteles.sort_values(by=["nombre"], inplace=True)
        print(hoteles.to_string(index=False))
       

## j. Mostrar la cantidad de hoteles de un destino que guste. 

# Dario: Aca tengo una duda, ¿!ue hago armo una funcion media especifica que me traiga la cuenta hecha desde Mongo a costa de perdida posible escalabilidad o me traigo el recorte de info de esa cuidad y hoteles y hago en la cuenta en pandas, ventajas mas facil de escala y codear para reutilizar en otras consultas(creo), desventajas  eficiencia

Segun chat gpt "💡 Regla general:

Para conteos, sumas, promedios y filtros simples, MongoDB siempre gana en eficiencia. Traer datos completos solo para contar es ineficiente y poco escalable."

In [28]:
## j. Mostrar la cantidad de hoteles de un destino que guste. 
agrupar= input("Introduce provincia o ciudad")
lugar= input("Introduce la provincia o ciudad ")

cursor = mongo.contador(
    nombre_base="clase",
    coleccion="hoteles",
    agrupacion=agrupar,
    campo_calculo="hotel_id",
#    filtrar={"provincia": provincia_elegida}
)


hoteles = pd.DataFrame(cursor)
hoteles


,_id,hotel_id
0,Ciudad Autónoma de Buenos Aires,2
1,La Pampa,2
2,Misiones,7
3,Chubut,1
4,Entre Ríos,13
5,La Rioja,5
6,San Juan,3
7,Santa Cruz,6
8,Catamarca,4
9,Córdoba,12


###### k. Mostrar las actividades de “Ushuaia” del tipo “aventura”.

###### l. Mostrar la cantidad de reservas concretadas de cada usuario. Mostrar el usuario y la cantidad 

#### Estadísticas

###### i.Destino más visitado

###### ii.Hotel más barato

###### iii.Actividad más popular.

#### Modificaciones

###### a.Incrementar el precio de las actividades de Tucuman en 5% 

###### b. Agregar al hotel id=1 el servicio de SPA 

###### c. Eliminar el destino que desee

###### d. Eliminar un usuario que desee 

###### e. Eliminar las relaciones AMIGO_DE para un usuario que quiera. 